In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math
from IPython.core.display import display,HTML
try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
    
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
import plotly
plotly.__version__

In [2]:
path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
df = pd.read_excel(open(path+'NICO_0511_anonymized.xlsx','rb'),na_values=["NAN","-1"],names = ['folder_id', 'user_id', 'email_domain', 'num_adds', 'num_edits', 'num_deletes', 'num_folder_members', 'major_content_type' ,'major_content_ext',  'age_folder_days' ])



In [19]:
for item in df.email_domain.unique():
    print item,"   "

*montana.edu subdomain    
*tamu.edu subdomain    
*uchc.edu subdomain    
*ucla.edu subdomain    
*ucsd.edu subdomain    
*ucsf.edu subdomain    
*uio.no subdomain    
*umn.edu subdomain    
*unm.edu subdomain    
*usc.edu subdomain    
*waseda.ac.jp subdomain    
2014.hull.ac.uk    
2017.icam.fr    
aa.ufl.edu    
aacc.edu    
aalto.fi    
aamu.edu    
aaschool.ac.uk    
aau.at    
aauj.edu    
abbey.bac.edu    
abdn.ac.uk    
aberdeen.ac.uk    
abet.org    
ac.upc.edu    
acad.umass.edu    
academicimpressions.com    
academyart.edu    
acadiau.ca    
acc.org    
accountancy.smu.edu.sg    
ace.tamut.edu    
aces.edu    
acg.edu    
acomedu.org    
acsw.ru.nl    
actx.edu    
acu.edu    
acu.edu.au    
ad.nmsu.edu    
ad.ucsd.edu    
ad.ufl.edu    
ad.umanitoba.ca    
ad.unc.edu    
adams.edu    
adelaide.edu.au    
adelphi.edu    
adjuncts.belmont.edu    
adler.edu    
adm.monash.edu.au    
admin.cam.ac.uk    
admin.fsu.edu    
admin.rochester.edu    
admin.umass.edu    
admin.usf.e

In [32]:
df[df.user_id == 8151657]#2174241653] #2335433869]

#table[table.column_name == some_value]

,folder_id,user_id,email_domain,num_adds,num_edits,num_deletes,num_folder_members,major_content_type,major_content_ext,age_folder_days
1,2174241653,8151657,*tamu.edu subdomain,256,272,62,2,office_docs,docx,512
2,2476590757,8151657,*tamu.edu subdomain,240,408,108,4,office_docs,docx,322
4,2748274007,8151657,*tamu.edu subdomain,134,344,20,2,office_docs,docx,138
5,434776253,8151657,*tamu.edu subdomain,3536,14604,2268,4,office_docs,docx,1573
6,455530165,8151657,*tamu.edu subdomain,6076,28802,3798,4,pdf,pdf,1549
8,1493785393,8151657,*tamu.edu subdomain,4508,11432,1602,4,design,eps,910
10,181065769,8151657,*tamu.edu subdomain,4524,156,1092,20,office_docs,doc,1943
11,1992175155,8151657,*tamu.edu subdomain,2238,498,84,3,other_docs,tex,619


In [7]:
flag_GC="m"  # yes: only plot the GC;    no: i plot the entire network,   m: i select multiple big connected components for plotting

Num_con_comp=3


#focus_university="harvard.edu"           #### harvard   missouri      northwestern   berkeley  ucla.  cambridge    ox.ac.uk
list_focus_univ=["missouri.edu","harvard.edu","northwestern.edu", "berkeley","ucla.","cambridge","ox.ac.uk","ucdavis","princeton","uchicago","umass","columbia","unizar.es","duke.edu"]
#list_focus_univ=["stanford"]
# list_focus_univ=["berkeley"]


G=nx.Graph()

dict_folder_list_domains={}
dict_folder_list_users={}
dict_user_id_domain={}
dict_user_id_edit={}
dict_user_id_del={}
dict_user_id_add={}
dict_user_id_age={}



print df.shape
cont=0
for row in df.iterrows():
        #print row[1]
    folder=row[1]["folder_id"]
    user_id=row[1]["user_id"]
        #print folder_id, user_id
                        
    email=row[1]["email_domain"]
    
    edits=row[1]["num_edits"]
    adds=row[1]["num_adds"]
    deletes=row[1]["num_deletes"]
   
    age=row[1]["age_folder_days"]
   
        
    flag_useful_line=0
    for focus_university in list_focus_univ:                 
        if focus_university in email:
            flag_useful_line =1
            break
                 
    if flag_useful_line ==1:
        try: 
            dict_folder_list_users[folder].append(user_id)
        except:
            dict_folder_list_users[folder]=[]
            dict_folder_list_users[folder].append(user_id)

        try:    
            dict_folder_list_domains[folder].append(email)
        except:            
            dict_folder_list_domains[folder]=[]
            dict_folder_list_domains[folder].append(email)

            
            
            
            
        dict_user_id_domain[user_id]=email     
        try:
            dict_user_id_edit[user_id] +=edits   # ojo!! the records are by folder and by user, so i need to accumulate!
            dict_user_id_del[user_id] +=deletes
            dict_user_id_add[user_id] +=adds 

            dict_user_id_age[user_id].append(age) 

        except :
            dict_user_id_edit[user_id] =edits
            dict_user_id_del[user_id] =deletes
            dict_user_id_add[user_id] =adds 

            dict_user_id_age[user_id]=[]
            dict_user_id_age[user_id].append(age) 

        
        
    cont +=1
    if cont % 10000 ==0:
        print cont
            
print "done with the dict., size:",len(dict_folder_list_users)





for user in  dict_user_id_age:
    dict_user_id_age[user]=np.median(dict_user_id_age[user])



for folder in dict_folder_list_users:
    dict_folder_list_users[folder]=list(set(dict_folder_list_users[folder]))  # remove possible duplicates

    dict_folder_list_domains[folder]=list(set(dict_folder_list_domains[folder]))
    
    
    lista=dict_folder_list_users[folder]
  
    if len(lista)>1:
        
        lista_pares=itertools.combinations(lista, 2)        
        
        for item in lista_pares:
            e1=item[0]
            e2=item[1]
            G.add_edge(e1,e2)
           # print "added edge:",e1, e2
#     else:   # FOR NOW, I DO NOT ADD THE NODES WITH COLLABORATORS OUTSIDE THE UNIVERSITY DOMAINS
#         user_id=lista[0]
        
#         G.add_node(user_id)




print "  N:", len(G.nodes()),"  L:", len(G.edges())    


GC_string=""
if flag_GC =="yes":   # i will plot only the GC
    print "selecting only the GC for plotting"
    G = max(nx.connected_component_subgraphs(G), key=len)
    print "  N:", len(G.nodes()),"  L:", len(G.edges())    
    GC_string="_GC"

elif flag_GC =="m":  # i will plot multiple large connected components
    
    print "selecting GC and some other large connected components for plotting"
   
    new_G = max(nx.connected_component_subgraphs(G), key=len)
    print "  N:",len(new_G.nodes()),"  L:",len(new_G.edges())
    lista_comp= reversed(sorted(nx.connected_component_subgraphs(G), key=len))
    
    
    cont_comp=0
    for comp in lista_comp:
#         cont_comp +=1
#         new_G.add_nodes_from(comp.nodes())
#         new_G.add_edges_from(comp.edges())
#         print cont_comp,"  N:",len(new_G.nodes()),"  L:",len(new_G.edges())
#         if cont_comp >=Num_con_comp:
#             break
            
            
        if len(comp.nodes())>=10:
            new_G.add_nodes_from(comp.nodes())
            new_G.add_edges_from(comp.edges())
            print "  N:",len(new_G.nodes()),"  L:",len(new_G.edges())


            
    
    G= new_G.copy()
    GC_string="_multiple_comp"
            
    
elif flag_GC =="no":  # i will plot the entire structure
    print "selecting entire network for plotting"
    print focus_university, "done.  N:", len(G.nodes()),"  L:", len(G.edges())
    GC = max(nx.connected_component_subgraphs(G), key=len)
    print "\n  GC:     N:", len(GC.nodes()), "  L:", len(GC.edges())



     

#### the master dict is larger than G.nodes(), and the set_node_attribute does not work in that case, so i get a selection of the dict only with the common nodes:
#######################
print len(G.nodes()), len(dict_user_id_domain)

new_dict_user_id = {k: dict_user_id_domain[k] for k in set(G.nodes()) & set(dict_user_id_domain.keys())}
print len(G.nodes()), len(new_dict_user_id)
# i add the email as node attributes    
nx.set_node_attributes(G, 'email', new_dict_user_id)
############



new_dict_user_id = {k: dict_user_id_edit[k] for k in set(G.nodes()) & set(dict_user_id_edit.keys())}
# i add the number of edits as node attributes    
nx.set_node_attributes(G, 'edits', new_dict_user_id)
############



new_dict_user_id = {k: dict_user_id_del[k] for k in set(G.nodes()) & set(dict_user_id_del.keys())}
# i add the number of deletes as node attributes    
nx.set_node_attributes(G, 'deletes', new_dict_user_id)
############



new_dict_user_id = {k: dict_user_id_add[k] for k in set(G.nodes()) & set(dict_user_id_add.keys())}
# i add the number of additions as node attributes    
nx.set_node_attributes(G, 'adds', new_dict_user_id)
############
     
    
 
new_dict_user_id = {k: dict_user_id_age[k] for k in set(G.nodes()) & set(dict_user_id_age.keys())}
# i add the median age as node attributes 
nx.set_node_attributes(G, 'median_age', new_dict_user_id)
############
 
    
    
    
    
    
    
    
       

print list_focus_univ, "done.  N:", len(G.nodes()),"  L:", len(G.edges())  
   
    
    
    
    
    
   





###### I get a different node shape for each domain (and i group all subdomains together)
dict_domains_cont={}
cont=0
for node in G.nodes():    
    
    domain=str(G.node[node]['email'])
    for focus_university in list_focus_univ:   
        if focus_university in domain:
            domain=focus_university
            break
    
    if domain in dict_domains_cont:
        G.node[node]['shape']=dict_domains_cont[domain]
    else:
        cont +=1
        dict_domains_cont[domain]=cont
        G.node[node]['shape']=dict_domains_cont[domain]


        

print "domains:",set(dict_domains_cont.keys()),"   shapes",set(dict_domains_cont.values())

(497553, 10)
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
done with the dict., size: 22538
  N: 21076   L: 68731
selecting GC and some other large connected components for plotting
  N: 2442   L: 10017
  N: 2442   L: 10017
  N: 3523   L: 15156
  N: 3773   L: 16476
  N: 3975   L: 17882
  N: 4174   L: 18883
  N: 4366   L: 22739
  N: 4549   L: 23303
  N: 4729   L: 24672
  N: 4832   L: 27089
  N: 4930   L: 27367
  N: 5019   L: 27884
  N: 5107   L: 28112
  N: 5193   L: 28373
  N: 5273   L: 30876
  N: 5350   L: 31218
  N: 5426   L: 32334
  N: 5499   L: 32878
  N: 5570   L: 33129
  N: 5626   L: 33264
  N: 5681   L: 33537
  N: 5736   L: 33696
  N: 5789   L: 34767
  N: 5841   L: 35932
  N: 5891   L: 36092
  N: 

In [12]:


########### i relabel the nodes so the plotly works later (otherwise, it doesnt work!)
mapping={}
cont=0
for n in G.nodes():
    mapping[n]=cont
    cont +=1
    
G=nx.relabel_nodes(G,mapping, copy=False)
print "nodes relabeled"
##############################




##### i get a layout for the plotting of the network
layt={}

layt=nx.fruchterman_reingold_layout(G,dim=3)     
#dict_pos=nx.fruchterman_reingold_layout(G,dim=2)
#dict_pos=nx.spectral_layout(G,dim=2)
#dict_pos=nx.spring_layout(G, dim=2)


    ##### layout options for   graphviz_layout:
    # dot - "hierarchical" or layered drawings of directed graphs. This is the default tool to use if edges have directionality.
    # neato - "spring model'' layouts.  This is the default tool to use if the graph is not too large (about 100 nodes) and you don't know anything else about it. Neato attempts to minimize a global energy function, which is equivalent to statistical multi-dimensional scaling.
    # fdp - "spring model'' layouts similar to those of neato, but does this by reducing forces rather than working with energy.
    # sfdp - multiscale version of fdp for the layout of large graphs.
    # twopi - radial layouts, after Graham Wills 97. Nodes are placed on concentric circles depending their distance from a given root node.
#     circo - circular layout, after Six and Tollis 99, Kauffman and Wiese 02. This is suitable for certain diagrams of multiple cyclic structures, such as certain telecommunications networks.

# layt=graphviz_layout(G, prog='fdp')

#### i save the resulting  layout as attributes of the nodes themselves
nx.set_node_attributes(G, 'pos', layt)

print "done getting the postion dict for",len(layt),"nodes"


nodes relabeled
done getting the postion dict for 2442 nodes


In [13]:
colors_as="age"  ## or "edits"  or age or k

##### i create the list for the info that will show up when hovering over a node, as well as for giving each node a color-coded, shape and size
list_values_colors=[]
list_labels=[]
list_sizes=[]
list_symols=[]
for node in G.nodes():
    if colors_as =="k":
        list_values_colors.append(G.degree(node))
        title_bar="degree"
    elif colors_as == "edits" :
        list_values_colors.append(G.node[node]['edits'])
        title_bar="number edits"
    elif colors_as == "age" :
        list_values_colors.append(G.node[node]['median_age'])
        title_bar="median lifespan of user's activity"
        
         
        
        
        
    
    #list_sizes.append(random.choice([1,2,3,4,5,6,7,8,9,10]))
    list_sizes.append(int(math.log(G.node[node]['edits']+0.001)+1))

    list_symols.append(G.node[node]['shape'])
                                  
    label=G.node[node]['email']+"  degree: "+str(G.degree(node))+"<br>ed: "+ str(G.node[node]['edits'])+"  adds: "+ str(G.node[node]['adds'])+"  del: "+ str(G.node[node]['deletes'])+"<br>median lifespan: "+str(G.node[node]['median_age'])
    list_labels.append(label)

    
    ## for the label, if i want a new line, code it as:   <br>  not as  \n
    
    # see complete list of symbols for nodes at:         https://plot.ly/matlab/reference/#scatter-marker  and    https://plot.ly/~neda/1032/hover-over-the-marker-points-to-see-their-names/
   

In [14]:
####### Set data for the Plotly plot of the graph:
if len(list_focus_univ)==1:
    plot_title=str(list_focus_univ).replace("[","").replace("]","").replace(",","-").replace('"','').title()
else:
    plot_title="multiple universities"


N=len(G.nodes())
L=len(G.edges())

Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in G.edges():
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]
    
    
    
    
    
trace1=Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=Line(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
trace2=Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=Marker(symbol='dot',
                             size=list_sizes, #6,
                             color=list_values_colors,
                             colorscale='Viridis',
                             line=Line(color='rgb(50,50,50)', width=0.5),                                                                                       
                             colorbar=dict(
                                thickness=15,
                                title=title_bar,#title='degree',
                                xanchor='left',
                                titleside='right'),                                
                             ),
               text=list_labels,
               hoverinfo='text'
               )
    
    
    
    
axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )





layout = Layout(
         title=plot_title,
         width=1000,
         height=1000,
         showlegend=False,
         scene=Scene(
         xaxis=XAxis(axis),
         yaxis=YAxis(axis),
         zaxis=ZAxis(axis),
        ),
     margin=Margin(
        t=100
    ),
    hovermode='closest',
    annotations=Annotations([
           Annotation(
           showarrow=False,
            text="Data source: Dropbox",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=Font(
            size=14
            )
            )
        ]),    )

print "done"

done


In [13]:
print "N:", len(G.nodes()), "L:", len(G.edges())

N: 932 L: 3778


In [15]:
data=Data([trace1, trace2])
fig=Figure(data=data, layout=layout)

iplot(fig)

In [16]:
# save the image as png and as html (interactive)
GC_string="_GC"
filename=str(list_focus_univ).replace("[","").replace("]","").replace(",","-").replace('"','')+GC_string
#py.image.save_as(fig, filename='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/'+filename+'_color_'+colors_as+GC_string+'_3D_.png',scale=5)  #scale controls the resolution of the image

offline.plot(fig, filename='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/'+filename+'_color_'+colors_as+GC_string+'_3D_.html')



"file:///home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/'missouri.edu'- 'harvard.edu'- 'northwestern.edu'- 'berkeley'- 'ucla.'- 'cambridge'- 'ox.ac.uk'- 'ucdavis'- 'princeton'- 'uchicago'- 'umass'- 'columbia'- 'unizar.es'- 'duke.edu'_GC_color_age_GC_3D_.html"

In [16]:
# help(py.image)

In [17]:
# with open(path+"first_network_"+focus_university+".pickle",'wb') as f:
#     %time    pickle.dump(G, f)
# print "written:",path+"first_network_"+focus_university+".pickle\n"




# with open(path+"dict_position_first_network_"+focus_university+".pickle",'wb') as f:
#     %time    pickle.dump(dict_pos, f)
# print "written:",path+"dict_position_first_network_"+focus_university+".pickle"
